In [1]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0)

def show_tensor_images(image_tensor, num_images=25, size=(1, 28, 28)):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in an uniform grid.
    '''
    # image_shifted = (image_tensor + 1) / 2\
    image_shifted = image_tensor
    image_unflat = image_shifted.detach().cpu().view(-1, *size)
    image_grid = make_grid(image_unflat[:num_images], nrow=4)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self , inn ) :
        self.ContractingBlock = nn.Sequential(
            nn.Conv2d(inn , 2*inn , kernel_size=(3,3)), nn.ReLu() ,
            nn.Conv2d(2*inn , 2*inn , kernel_size=(3,3)) , nn.ReLu(), 
            nn.MaxPool2d()
        )
    def forward(self , images):
        return self.ContractingBlock(images)


In [2]:
class DecoderBlock(nn.Module):
    def __init__(self , inn ):
        self.upsampling = nn.Sequential(
            nn.Upsample(scale_factor=2 , mode = 'bilinear' , align_corners=True) , 
            nn.Conv2d(inn , inn//2 , kernel_size=(2,2)) ,nn.ReLU() , 
        )
        
        self.ExpandingBlock = nn.Sequential(
            nn.Conv2d(inn , inn//2 , kernel_size=(3,3))  , nn.ReLU(),
            nn.Conv2d(inn//2 , inn//4 , kernel_size= (3,3)) , nn.ReLU()
        )

    def crop_features(self ,image , new_shape):

        x  , y = image.shape[2] - new_shape[2]   , image.shape[3] -new_shape[3]
        x,y= x//2 , y // 2
        image = image[...,x:x+new_shape[2],y:y+new_shape[3]]

        return image

    def forward(self , image ,encoder_feature):
        x = self.upsampling(image)
        x = torch.cat([x , crop_features(encoder_feature)] , axis = 1)
        x = self.ExpandingBlock(x)

        return x





In [3]:
class final_head(nn.Module):
    def __init__(self , inn , out):
        self.map = nn.Conv2d(inn , out , kernel_size=(1,1))
    def forward(self , x):
        return self.map(x)